In [ ]:
# prompt: attach gdrive to save env and its files

from google.colab import drive
drive.mount('/content/gdrive/')



Mounted at /content/gdrive/


In [ ]:
!pwd
!python --version
!which python


/content
Python 3.10.12
/usr/local/bin/python


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "datasetOriginal.py"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# import pandas as pd
# import json

# # Read the JSON file
# with open('/content/gdrive/MyDrive/ema/archive/wikisql_train.json', 'r') as f:
#     data = json.load(f)

# # Convert to a Pandas DataFrame
# df = pd.DataFrame(data)
# print(df.head())

# # Save as a Parquet file
# df.to_parquet('/content/gdrive/MyDrive/ema/archive/datasetOriginal.parquet')

                                            question  \
0    Tell me what the notes are for South Australia    
1  What is the current series where the new serie...   
2            What is the format for South Australia?   
3  Name the background colour for the Australian ...   
4      how many times is the fuel propulsion is cng?   

                                              answer  \
0  SELECT Notes FROM table WHERE Current slogan =...   
1  SELECT Current series FROM table WHERE Notes =...   
2  SELECT Format FROM table WHERE State/territory...   
3  SELECT Text/background colour FROM table WHERE...   
4  SELECT COUNT Fleet Series (Quantity) FROM tabl...   

                                 id  
0  fc76a9f04e884afe8947b8acc9e0900d  
1  fc347c56706640a598453876af590b76  
2  e7240d86ea4940e181db5a75166fe83b  
3  c0f09c579d424fcc81a416ba40e5ed75  
4  4a5cffa993eb4df2b97365aea8808365  


In [ ]:
dataset = load_dataset('/content/gdrive/MyDrive/ema/archive/datasetOriginal.py')

print(dataset)

The repository for datasetOriginal contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/datasetOriginal.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 56355
    })
})


In [ ]:
def preprocess_function(examples):
    return {"text": "<s>[INST]" + examples['question'] + "[/INST]" + examples["answer"] + "</s>"}

dataset = dataset.map(preprocess_function, remove_columns=['question', 'answer'])
print(dataset['train'].column_names)  # Should print ['text']


Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

['text']


In [ ]:
import torch

print("Is CUDA available?", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0))


Is CUDA available? True
CUDA device count: 1
CUDA device name: Tesla T4


In [ ]:
# Load dataset (you can process it here)
# dataset = load_dataset('dataset.py')

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,4.639200
50,5.568700
75,3.725400
100,3.170600
125,1.952000
150,1.647300
175,1.508100
200,1.392600
225,1.467200
250,1.339300


Step,Training Loss
25,4.639200
50,5.568700
75,3.725400
100,3.170600
125,1.952000
150,1.647300
175,1.508100
200,1.392600
225,1.467200
250,1.339300


In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
print("Hello, World!")

Hello, World!


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Tell me what the notes are for South Australia"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
result = pipe(f"{prompt} ")
print(result[0]['generated_text'])

Tell me what the notes are for South Australia 1 SELECT Notes FROM table WHERE State = south australia 1 AND Region = south australia AND Year = 1999 AND Notes = south australia 1 AND State = south australia AND Region = south australia AND Year = 1999 AND Notes = south australia 1 AND State = south australia AND Region = south australia AND Year = 1999 AND Notes = south australia 1 AND State = south australia AND Region = south australia AND Year = 1999 AND Notes = south australia 1 AND State = south australia AND Region = south australia AND Year = 1999 AND Notes = south australia 1 AND State = south australia AND Region = south australia AND Year = 1999 AND Notes = south australia 1 AND State = south australia AND Region = south australia AND Year =


In [ ]:
# Run text generation pipeline with our next model
prompt = "What rank was Bryan Danielson in this chart?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
result = pipe(f"{prompt} ")
print(result[0]['generated_text'])

What rank was Bryan Danielson in this chart?  SELECT Rank FROM table WHERE Name = bryan danielson AND Chart = top 50 AND Year = 2007 AND Name = bryan danielson AND Chart = top 50 AND Year = 2007 AND Name = bryan danielson AND Chart = top 50 AND Year = 2007 AND Name = bryan danielson AND Chart = top 50 AND Year = 2007 AND Name = bryan danielson AND Chart = top 50 AND Year = 2007 AND Name = bryan danielson AND Chart = top 50 AND Year = 2007 AND Name = bryan danielson AND Chart = top 50 AND Year = 2007 AND Name = bryan danielson AND Chart = top 50


In [ ]:
# Run text generation pipeline with our next model
prompt = "How much marks did Ratan score in his maths test?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
result = pipe(f"{prompt} ")
print(result[0]['generated_text'])

How much marks did Ratan score in his maths test?  SELECT Maths FROM table WHERE Name = Ratan AND Subject = maths AND Marks = 80 AND Test = maths test AND Test Name = maths test AND Subject = maths AND Subject = maths AND Test = maths test AND Name = ratan AND Marks = 80 AND Test = maths test AND Subject = maths AND Test = maths test AND Name = ratan AND Marks = 80 AND Test = maths test AND Subject = maths AND Test = maths test AND Name = ratan AND Marks = 80 AND Test = maths test AND Subject = maths AND Test = maths test AND Name = ratan AND Marks = 80 AND Test = maths test AND Subject = maths AND Test = maths test AND Name = ratan AND Marks = 80


In [ ]:
# Run text generation pipeline with our next model
prompt = "where does Ratan Prakash belong to?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
result = pipe(f"{prompt} ")
print(result[0]['generated_text'])

where does Ratan Prakash belong to?  SELECT Home town FROM table WHERE Player = Ratan Prakash AND Position = Wicket-keeper AND Country = India AND First-class = India AND Test = India AND ODI = India AND T20I = India AND IPL = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic = India AND International = India AND Domestic


In [ ]:
# Run text generation pipeline with our next model
prompt = "where is new delhi situated?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
result = pipe(f"{prompt} ")
print(result[0]['generated_text'])

where is new delhi situated?  SELECT Location FROM table WHERE Name = New Delhi AND Country = India AND Capital = New Delhi AND Population = 16,753,500 AND City = New Delhi AND State = Delhi AND Time zone = IST AND Latitude = 28.6172° N AND Longitude = 77.1289° E AND Population (2011) = 16,753,500 AND Area (km 2 ) = 1,484 AND Population (2011) (millions) = 16,753,500 AND City = New Delhi AND State = Delhi AND Time zone = IST AND Latitude = 28.6172° N AND Longitude = 77.1289° E AND Population (2011) (mill


In [ ]:
# Run text generation pipeline with our next model
prompt = "what is the nationality of donal trump?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# result = pipe(f"<s>[INST] {prompt} [/INST]")
result = pipe(f"{prompt} ")
print(result[0]['generated_text'])

what is the nationality of donal trump?  SELECT Nationality FROM table WHERE Name = Donal Trump AND Family = Trump family AND Gender = Male AND Age = 70 AND Position = President AND Party = Republican AND Religion = Presbyterian AND Spouse = Marla Maples AND Children = Ivanka Trump, Donald Trump Jr., Eric Trump, Tiffany Trump AND Net worth = $300 million AND Inauguration = January 20, 2017 AND First Lady = Melania Trump AND Vice President = Mike Pence AND Cabinet = John Kelly, Rex Tillerson, James Mattis, Jeff Sessions, Mike Pompeo, Tom Price, Ryan Zinke, Elaine Chao, Wilbur Ross, and Betsy DeVos AND Previous office = New York City Mayor AND Previous party = Republican AND Previous position = Mayor AND Previous spouse = Marla Maples AND
